<a href="https://colab.research.google.com/github/wesol05/dw_matrix_road_sign/blob/master/matrix3_day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
cd /content/drive/My Drive/Colab Notebooks/matrix_three/data

/content/drive/My Drive/Colab Notebooks/matrix_three/data


In [2]:
!pip install hyperopt

In [13]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure
from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
train = pd.read_pickle('train.p')
test = pd.read_pickle('test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1 : y_train = to_categorical(y_train)
if y_test.ndim == 1 : y_test = to_categorical(y_test)
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir=os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensorboard_callback]

  )

  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def train_and_predict(model):
  model_trained = train_model(model, X_train, y_train)

  return predict(model_trained, X_test, y_test)

In [12]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential((
      Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
      Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
      MaxPool2D(),
      Dropout(0.3),

      Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
      Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
      MaxPool2D(),
      Dropout(0.3),

      Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
      Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
      MaxPool2D(),
      Dropout(0.3),

      Flatten(),

      Dense(1024, activation='relu'),
      Dropout(0.3),

      Dense(1024, activation='relu'),
      Dropout(0.3),

      Dense(num_classes, activation='softmax')
  ))

train_and_predict(get_cnn_v5(input_shape, num_classes))


Epoch 1/5
272/272 [==============================] - 4s 14ms/step - loss: 2.6056 - accuracy: 0.2897 - val_loss: 1.0165 - val_accuracy: 0.6813
Epoch 2/5
272/272 [==============================] - 4s 13ms/step - loss: 0.8375 - accuracy: 0.7305 - val_loss: 0.2777 - val_accuracy: 0.9064
Epoch 3/5
272/272 [==============================] - 4s 13ms/step - loss: 0.4337 - accuracy: 0.8631 - val_loss: 0.1298 - val_accuracy: 0.9630
Epoch 4/5
272/272 [==============================] - 4s 13ms/step - loss: 0.2882 - accuracy: 0.9099 - val_loss: 0.0720 - val_accuracy: 0.9791
Epoch 5/5
272/272 [==============================] - 4s 13ms/step - loss: 0.2220 - accuracy: 0.9322 - val_loss: 0.0579 - val_accuracy: 0.9831


0.9562358276643991

In [0]:
def get_model(params):
  return Sequential((
      Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
      Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_one']),

      Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
      Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_two']),

      Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
      Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_three']),

      Flatten(),

      Dense(1024, activation='relu'),
      Dropout(params['dropout_dense_block_one']),

      Dense(1024, activation='relu'),
      Dropout(params['dropout_dense_block_two']),

      Dense(num_classes, activation='softmax')
  ))

def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0
  )

  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accurany={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [24]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 200.0, 'dropout_cnn_block_one': 0.41062588780286247, 'dropout_cnn_block_three': 0.3832998907384372, 'dropout_cnn_block_two': 0.4401518471947107, 'dropout_dense_block_one': 0.603385234932299, 'dropout_dense_block_two': 0.6766750715367547}
accurany=0.7936508059501648
{'batch_size': 100.0, 'dropout_cnn_block_one': 0.32100969118238787, 'dropout_cnn_block_three': 0.39210965218717364, 'dropout_cnn_block_two': 0.43660957256536226, 'dropout_dense_block_one': 0.6869941576972738, 'dropout_dense_block_two': 0.6676227563024388}
accurany=0.9306122660636902
{'batch_size': 130.0, 'dropout_cnn_block_one': 0.3339205432410491, 'dropout_cnn_block_three': 0.3376286788572822, 'dropout_cnn_block_two': 0.30856077369924534, 'dropout_dense_block_one': 0.683082511137193, 'dropout_dense_block_two': 0.6213209686035601}
accurany=0.9621315002441406
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.41844648714334315, 'dropout_cnn_block_three': 0.4517537645474782, 'dropout_cnn_block_two': 0.421004517588